<a href="https://colab.research.google.com/github/Sabrina-Hendricks/DS4002-Group13-Project3/blob/main/Scripts/Project3MI3Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get photos from Github

In [1]:
import pandas as pd

# Define the base URL for your GitHub repository
base_url = 'https://raw.githubusercontent.com/Sabrina-Hendricks/DS4002-Group13-Project3/main/Data'

# List of breeds (directories)
breeds = ['Beagle', 'Boxer', 'Bulldog', 'Chihuahua', 'Chow', 'CockerSpaniel',
          'Doberman', 'GermanShepherd', 'Golden', 'GreatDane', 'Husky', 'Lab',
          'Pomeranian', 'Pug', 'Rottweiler', 'SaintBernard', 'Shih-tzu', 'StandardPoodle', 'StandardSchnauzer', 'Whippet']  # Add other breeds as needed

# Initialize lists to store the image URLs and labels
file_urls = []
labels = []

# Construct URLs for each breed
for breed in breeds:
    for i in range(1, 101):
        file_name = f"{breed}_{i}.jpg"  # Assumes file names follow this convention, e.g., 'Beagle_1.jpg'
        file_url = f"{base_url}/{breed}/{file_name}"

        file_urls.append(file_url)
        labels.append(breed)

# Create a DataFrame with the URLs and labels
df = pd.DataFrame({
    'file_url': file_urls,
    'label': labels
})

# Display the DataFrame
df.head()

,file_url,label
0,https://raw.githubusercontent.com/Sabrina-Hend...,Beagle
1,https://raw.githubusercontent.com/Sabrina-Hend...,Beagle
2,https://raw.githubusercontent.com/Sabrina-Hend...,Beagle
3,https://raw.githubusercontent.com/Sabrina-Hend...,Beagle
4,https://raw.githubusercontent.com/Sabrina-Hend...,Beagle


Load image from urls

In [6]:
import numpy as np
import requests
from PIL import Image
from io import BytesIO

# Function to load and preprocess an image from a URL
def load_image_from_url(url, target_size=(128, 128)):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')  # Ensure 3 channels (RGB)
    img = img.resize(target_size)  # Resize to target size
    img_array = np.array(img) / 255.0  # Normalize to [0, 1] range
    return img_array

Prep data generator to get images from urls

In [7]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import to_categorical

# Create a mapping from breed labels to integers
breed_to_index = {breed: idx for idx, breed in enumerate(df['label'].unique())}
df['label_idx'] = df['label'].map(breed_to_index)

# Custom data generator to load images in batches from URLs
class URLImageDataGenerator(Sequence):
    def __init__(self, dataframe, batch_size=32, target_size=(128, 128), shuffle=True):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_data = self.dataframe.iloc[index * self.batch_size:(index + 1) * self.batch_size]

        # Load and preprocess images
        X = np.array([load_image_from_url(url, self.target_size) for url in batch_data['file_url']])
        y = to_categorical(batch_data['label_idx'].values, num_classes=len(breed_to_index))

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)

# Initialize the data generator
batch_size = 32
train_generator = URLImageDataGenerator(df, batch_size=batch_size, target_size=(128, 128))


Define model

In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the base model layers to retain pre-trained features
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(len(breed_to_index), activation='softmax')(x)

# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Train model

In [9]:
# Train the model
epochs = 10  # Adjust the number of epochs as needed

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs
)


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x7f8d050015d0>

Make predictions

In [ ]:
# Function to make predictions
def predict_breed(url):
    img_array = np.expand_dims(load_image_from_url(url), axis=0)  # Add batch dimension
    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_breed = [breed for breed, idx in breed_to_index.items() if idx == predicted_index][0]
    return predicted_breed

# Test the prediction
test_url = 'https://raw.githubusercontent.com/Sabrina-Hendricks/DS4002-Group13-Project3/main/Data/Beagle/Beagle_1.jpg'
print("Predicted breed:", predict_breed(test_url))
